In [ ]:
# basic
import os,datetime
# numerical 
import numpy as np
# visualisation
import matplotlib.pyplot as plt
# model_func
from tools.model_func import *

# get_cascaded_softmax

In [1]:
# imports
from scipy.special import softmax
import scipy.sparse as sp
import pickle

# function
def get_cascade_sm(y_preds,IN_DIR):
    child_to_siblings = pickle.load(open(os.path.join(IN_DIR,'child_to_siblings.pkl'),'rb'))
    parent_to_child = pickle.load(open(os.path.join(IN_DIR,'parent_to_child.pkl'),'rb'))
    cascade_sm = [softmax(y_preds[0],axis=1)]
    for i in range(len(y_preds)-1):
        data = []
        row_ind = []
        col_ind = []
        sms = cascade_sm[-1]
        for key,val in parent_to_child[i].items():
            child_sm = softmax(y_preds[i+1][:,val],axis=1)
            data.append(np.multiply(child_sm,sms[:,key,np.newaxis]).reshape(-1))
            row_ind.append(np.repeat(np.arange(sms.shape[0]),len(val)))
            col_ind.append(np.tile(val,sms.shape[0]))
        data = np.concatenate(data)
        row_ind = np.concatenate(row_ind)
        col_ind = np.concatenate(col_ind)
        cascade_sm.append(sp.csr_matrix((data,(row_ind,col_ind))).toarray())
    return cascade_sm

# get pAtk

In [ ]:
def get_pAtk(y_true,y_pred,k,per_label=False):
    y_true_sparse = np.argmax(np.asarray(y_true),axis=1)
    Atk = np.any(y_true_sparse[:,np.newaxis]==y_pred[:,:k],axis=1).astype(int)
    if per_label:
        lab = []
        for i in range(y.shape[-1]):
            lab.append(np.mean(Atk[y_true_sparse==i]))
        return np.array(lab)
    else:
        return np.mean(Atk)